# Text Summarization with DialogSum

**Module 02 | Notebook 3 of 3**

In this notebook, we'll fine-tune a sequence-to-sequence model for dialogue summarization.

## Learning Objectives

By the end of this notebook, you will be able to:
1. Work with encoder-decoder models (T5, BART)
2. Prepare data for seq2seq tasks
3. Use ROUGE metrics for evaluation
4. Generate summaries from dialogue

---

In [1]:
%%capture
!pip install transformers datasets accelerate evaluate rouge-score nltk

In [2]:
import torch
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset
import evaluate
import numpy as np
import nltk
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data for sentence tokenization
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


---

## Understanding Sequence-to-Sequence Models

### How Summarization Works

```
Input (Dialogue):
┌─────────────────────────────────────────────┐
│ Person A: Hi, how are you?                  │
│ Person B: I'm great! Just got promoted.     │
│ Person A: Congratulations! That's amazing!  │
│ Person B: Thanks! Let's celebrate tonight.  │
└─────────────────────────────────────────────┘
                    │
                    ▼
            ┌───────────────┐
            │    ENCODER    │  (Understands input)
            └───────────────┘
                    │
                    ▼
            ┌───────────────┐
            │    DECODER    │  (Generates output)
            └───────────────┘
                    │
                    ▼
Output (Summary):
┌─────────────────────────────────────────────┐
│ Person B got promoted and they plan to      │
│ celebrate tonight.                          │
└─────────────────────────────────────────────┘
```

### Popular Summarization Models

| Model | Parameters | Best For |
|-------|------------|----------|
| T5-small | 60M | Quick experiments |
| T5-base | 220M | Good balance |
| BART-base | 140M | News articles |
| FLAN-T5 | Various | Instruction-tuned |

---

## Load the DialogSum Dataset

In [3]:
# Load DialogSum dataset
dataset = load_dataset("knkarthick/dialogsum")

print("Dataset structure:")
print(dataset)
print(f"\nTrain examples: {len(dataset['train'])}")
print(f"Validation examples: {len(dataset['validation'])}")
print(f"Test examples: {len(dataset['test'])}")

README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

Train examples: 12460
Validation examples: 500
Test examples: 1500


In [4]:
# Explore the data
print("Sample dialogue and summary:")
print("=" * 60)
example = dataset['train'][0]

print("DIALOGUE:")
print(example['dialogue'])
print("\nSUMMARY:")
print(example['summary'])
print("\nTOPIC:")
print(example.get('topic', 'N/A'))

Sample dialogue and summary:
DIALOGUE:
#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.

SUMMARY:
Mr. Smith's getting

In [5]:
# Analyze lengths
dialogue_lengths = [len(d.split()) for d in dataset['train']['dialogue']]
summary_lengths = [len(s.split()) for s in dataset['train']['summary']]

print("Length Statistics (words):")
print(f"\nDialogues:")
print(f"  Mean: {np.mean(dialogue_lengths):.0f}")
print(f"  Max: {max(dialogue_lengths)}")

print(f"\nSummaries:")
print(f"  Mean: {np.mean(summary_lengths):.0f}")
print(f"  Max: {max(summary_lengths)}")

# Compression ratio
ratios = [s/d for s, d in zip(summary_lengths, dialogue_lengths) if d > 0]
print(f"\nCompression ratio: {np.mean(ratios):.1%}")

Length Statistics (words):

Dialogues:
  Mean: 131
  Max: 985

Summaries:
  Mean: 23
  Max: 153

Compression ratio: 18.5%


---

## Data Preparation for Seq2Seq

In [6]:
# Load model and tokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# T5 uses a prefix for summarization
prefix = "summarize: "

# Tokenization parameters
max_input_length = 512
max_target_length = 128

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    """Tokenize inputs and targets for seq2seq training."""
    # Add prefix to inputs
    inputs = [prefix + doc for doc in examples['dialogue']]
    
    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding=True
    )
    
    # Tokenize targets (summaries)
    labels = tokenizer(
        examples['summary'],
        max_length=max_target_length,
        truncation=True,
        padding=True
    )
    
    model_inputs['labels'] = labels['input_ids']
    
    return model_inputs

In [8]:
# Use smaller subsets for faster training
train_size = 1000
val_size = 200

train_data = dataset['train'].shuffle(seed=42).select(range(train_size))
val_data = dataset['validation'].shuffle(seed=42).select(range(val_size))

# Tokenize
train_tokenized = train_data.map(preprocess_function, batched=True)
val_tokenized = val_data.map(preprocess_function, batched=True)

print(f"Training samples: {len(train_tokenized)}")
print(f"Validation samples: {len(val_tokenized)}")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Training samples: 1000
Validation samples: 200


---

## Understanding ROUGE Metrics

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) measures summary quality:

| Metric | What it Measures |
|--------|------------------|
| ROUGE-1 | Unigram (word) overlap |
| ROUGE-2 | Bigram (2-word) overlap |
| ROUGE-L | Longest common subsequence |

Higher scores = better summaries

In [ ]:
# Load ROUGE metric
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    """
    Compute ROUGE metrics for summarization evaluation.
    
    Note on Token Handling:
    -----------------------
    When using Seq2SeqTrainer with predict_with_generate=True, the predictions 
    and labels arrays come as numpy int64 values. The HuggingFace Fast Tokenizer 
    (Rust backend) can throw an OverflowError when decoding these values directly.
    
    This happens because:
    1. Padding tokens are marked as -100 (a convention in HuggingFace for ignore_index)
    2. Numpy int64 values may not convert cleanly to Rust's integer types
    3. Token IDs outside the valid vocabulary range cause decoding issues
    
    The fix below explicitly:
    - Converts each token to a Python int using int(tok)
    - Validates tokens are within [0, vocab_size) range
    - Replaces invalid tokens (including -100) with pad_token_id
    """
    predictions, labels = eval_pred
    
    # Get pad token id and vocab size for validation
    pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
    vocab_size = tokenizer.vocab_size
    
    # Clean predictions: convert to Python int and replace invalid values
    predictions_clean = []
    for row in predictions:
        clean_row = []
        for tok in row:
            tok_int = int(tok)  # Explicitly convert to Python int
            if tok_int < 0 or tok_int >= vocab_size:
                clean_row.append(pad_id)
            else:
                clean_row.append(tok_int)
        predictions_clean.append(clean_row)
    
    decoded_preds = tokenizer.batch_decode(predictions_clean, skip_special_tokens=True)
    
    # Clean labels: convert to Python int and replace -100 padding
    labels_clean = []
    for row in labels:
        clean_row = []
        for tok in row:
            tok_int = int(tok)  # Explicitly convert to Python int
            if tok_int < 0 or tok_int >= vocab_size:
                clean_row.append(pad_id)
            else:
                clean_row.append(tok_int)
        labels_clean.append(clean_row)
    
    decoded_labels = tokenizer.batch_decode(labels_clean, skip_special_tokens=True)
    
    # Strip whitespace
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    
    # Compute ROUGE scores
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    
    return {
        'rouge1': result['rouge1'],
        'rouge2': result['rouge2'],
        'rougeL': result['rougeL']
    }

---

## Model Training

In [14]:
# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

total_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {total_params:,}")

Model parameters: 60,506,624


In [15]:
# Seq2Seq training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./summarization_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    
    # Generation during evaluation
    predict_with_generate=True,
    generation_max_length=max_target_length,
    
    # Evaluation
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    
    # Logging
    logging_steps=50,
    
    # Performance
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# Data collator for seq2seq
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

# Create trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [16]:
# Train
print("Starting training...")
print("=" * 50)
trainer.train()
print("\nTraining complete!")

Starting training...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,1.182600,0.866480,0.000000,0.000000,0.000000
2,0.631000,0.685007,0.174265,0.064189,0.148405
3,0.606800,0.671469,0.250832,0.090146,0.212441


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].



Training complete!


---

## Evaluation

In [17]:
# Evaluate
eval_results = trainer.evaluate()

print("Evaluation Results:")
print("=" * 40)
print(f"ROUGE-1: {eval_results['eval_rouge1']:.2%}")
print(f"ROUGE-2: {eval_results['eval_rouge2']:.2%}")
print(f"ROUGE-L: {eval_results['eval_rougeL']:.2%}")

Evaluation Results:
ROUGE-1: 25.08%
ROUGE-2: 9.01%
ROUGE-L: 21.24%


---

## Generate Summaries

In [18]:
from transformers import pipeline

# Create summarization pipeline
summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

Device set to use cuda:0


In [19]:
# Test on examples from the test set
test_examples = dataset['test'].select(range(3))

print("Generated Summaries:")
print("=" * 60)

for i, example in enumerate(test_examples):
    dialogue = example['dialogue']
    reference = example['summary']
    
    # Generate summary
    generated = summarizer(
        prefix + dialogue,
        max_length=128,
        min_length=20,
        do_sample=False
    )[0]['summary_text']
    
    print(f"\n--- Example {i+1} ---")
    print(f"\nDIALOGUE (truncated):")
    print(dialogue[:300] + "...")
    print(f"\nREFERENCE SUMMARY:")
    print(reference)
    print(f"\nGENERATED SUMMARY:")
    print(generated)
    print("-" * 60)

Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Summaries:


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Example 1 ---

DIALOGUE (truncated):
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office commu...

REFERENCE SUMMARY:
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.

GENERATED SUMMARY:
#Person1# is ready to take a dictation for her. She wants to keep the memo typed up and distributed to all employees.
------------------------------------------------------------


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Example 2 ---

DIALOGUE (truncated):
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office commu...

REFERENCE SUMMARY:
In order to prevent employees from wasting time on Instant Message programs, #Person1# decides to terminate the use of those programs and asks Ms. Dawson to send out a memo to all employees by the afternoon.

GENERATED SUMMARY:
#Person1# is ready to take a dictation for her. She wants to keep the memo typed up and distributed to all employees.
------------------------------------------------------------

--- Example 3 ---

DIALOGUE (truncated):
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are 

In [20]:
# Test on custom dialogue
custom_dialogue = """
#Person1#: Hey Sarah, have you finished the project report?
#Person2#: Almost done! Just need to add the conclusion section.
#Person1#: Great! The client meeting is tomorrow at 2 PM.
#Person2#: I'll have it ready by noon so you can review it.
#Person1#: Perfect. Also, can you include the budget projections?
#Person2#: Already added those. I also updated the timeline chart.
#Person1#: You're a lifesaver! Thanks so much.
#Person2#: No problem! See you tomorrow.
"""

summary = summarizer(
    prefix + custom_dialogue,
    max_length=100,
    min_length=20,
    do_sample=False
)[0]['summary_text']

print("Custom Dialogue Summarization:")
print("=" * 50)
print("\nINPUT:")
print(custom_dialogue)
print("\nSUMMARY:")
print(summary)

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Custom Dialogue Summarization:

INPUT:

#Person1#: Hey Sarah, have you finished the project report?
#Person2#: Almost done! Just need to add the conclusion section.
#Person1#: Great! The client meeting is tomorrow at 2 PM.
#Person2#: I'll have it ready by noon so you can review it.
#Person1#: Perfect. Also, can you include the budget projections?
#Person2#: Already added those. I also updated the timeline chart.
#Person1#: You're a lifesaver! Thanks so much.
#Person2#: No problem! See you tomorrow.


SUMMARY:
#Person1#'ll have the final report ready by noon so you can review it. She's a lifesaver!


---

## Generation Parameters

You can control how summaries are generated:

In [21]:
test_dialogue = dataset['test'][0]['dialogue']

# Different generation strategies
print("Generation Strategies Comparison:")
print("=" * 50)

# Greedy (default)
greedy = summarizer(prefix + test_dialogue, max_length=100, do_sample=False)[0]['summary_text']
print(f"\nGreedy: {greedy}")

# Beam search
beam = summarizer(prefix + test_dialogue, max_length=100, num_beams=4, do_sample=False)[0]['summary_text']
print(f"\nBeam Search (4): {beam}")

# Sampling
sampled = summarizer(prefix + test_dialogue, max_length=100, do_sample=True, top_k=50, temperature=0.7)[0]['summary_text']
print(f"\nSampling (temp=0.7): {sampled}")

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generation Strategies Comparison:


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Greedy: #Person1# is ready to take a dictation for her. She wants to keep the memo typed up and distributed to all employees before 4 pm.


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Beam Search (4): #Person1# is ready to take a dictation for her. She wants to keep the memo typed up and distributed to all employees before 4 pm.

Sampling (temp=0.7): #Person1# is ready to take a dictation for her. She wants to keep the memo typed up and distributed to all employees before 4 pm.


### Generation Parameters Explained

| Parameter | Effect |
|-----------|--------|
| `max_length` | Maximum output tokens |
| `min_length` | Minimum output tokens |
| `num_beams` | Beam search width (higher = better but slower) |
| `do_sample` | Enable random sampling |
| `temperature` | Randomness (0=deterministic, 1=default, >1=more random) |
| `top_k` | Limit sampling to top K tokens |
| `top_p` | Nucleus sampling threshold |

---

## 🎯 Student Challenge

### Challenge: Fine-tune for News Summarization

In [ ]:
# TODO: Adapt this notebook for news summarization
# 1. Load the CNN/DailyMail dataset: load_dataset("cnn_dailymail", "3.0.0")
# 2. The columns are 'article' (input) and 'highlights' (summary)
# 3. Update the preprocess_function
# 4. Train and evaluate
# 5. Compare ROUGE scores

# Hint: CNN/DailyMail articles are longer, you may need max_input_length=1024

# Your solution:


---

## Key Takeaways

1. **Seq2Seq models** have separate encoder and decoder components
2. **T5 uses prefixes** ("summarize:", "translate:") to specify tasks
3. **ROUGE metrics** measure n-gram overlap between generated and reference summaries
4. **Generation parameters** control output quality and diversity
5. **Beam search** usually produces better results than greedy decoding

---

## Next Steps

Continue to Module 03: **Model Optimization**
- `03_Model_Optimization/01_intro_to_optimization.ipynb`